#                    WEB SCRAPING ASSIGNMENT - 3

In [ ]:
!pip install selenium

In [ ]:
# 1.Q. Write a python program which searches all the product under a particular product from www.amazon.in. The 
#product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
#guitars.

In [ ]:
import selenium
from selenium import webdriver
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import warnings

warnings.filterwarnings("ignore")
import time

def search_amazon_product_with_selenium(product_name):
    driver=webdriver.Chrome() 
    try:
        
        driver.get("https://www.amazon.in/")
        search_input = driver.find_element(By.ID, "nav-bb-search")
        search_input.send_keys(product_name)
        search_input.submit()
        time.sleep(5)
        print(search_input.text)
    except NoSuchElementException as e:
        print("Exception Raised : ",e)
        search_input = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div[2]/form/div/div/input')
        print(search_input.text)
        driver.close()
        

if __name__ == "__main__":
    product_name = input("Enter the product you want to search for on Amazon.in: ")
    search_amazon_product_with_selenium(product_name)


In [ ]:
# 2.Q. In the above question, now scrape the following details of each product listed in first 3 pages of your search
#results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
#scrape all the products available under that product name. Details to be scraped are: "Brand 
#Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and 
#“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“. 

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def scrape_amazon_product_details(product_name):
    # Initialize a Chrome WebDriver
    driver = webdriver.Chrome()

    try:
        product_data = []
        for page in range(1, 4): 
            url = f"https://www.amazon.in/s?k={product_name.replace(' ', '+')}&page={page}"
            driver.get(url)

        
            wait = WebDriverWait(driver, 10)
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.s-result-item")))

            product_results = driver.find_elements(By.CSS_SELECTOR, "div.s-result-item")
            if not product_results:
                break  

            for product in product_results:
                try:
                    brand_name = product.find_element(By.CSS_SELECTOR, "div.a-row.a-size-base.a-color-secondary span.a-text-normal").text
                except NoSuchElementException:
                    brand_name = "-"

                try:
                    product_name = product.find_element(By.CSS_SELECTOR, "span.a-text-normal").text
                except NoSuchElementException:
                    product_name = "-"

                try:
                    price = product.find_element(By.CSS_SELECTOR, "span.a-price-whole").text
                except NoSuchElementException:
                    price = "-"

                try:
                    return_exchange = product.find_element(By.CSS_SELECTOR, "div.a-row.a-size-small span.s-icon-label").text
                except NoSuchElementException:
                    return_exchange = "-"

                try:
                    expected_delivery = product.find_element(By.CSS_SELECTOR, "div.a-row span.a-text-bold").text
                except NoSuchElementException:
                    expected_delivery = "-"

                try:
                    availability = product.find_element(By.CSS_SELECTOR, "div.a-row span.a-size-base").text
                except NoSuchElementException:
                    availability = "-"

                try:
                    product_url = product.find_element(By.CSS_SELECTOR, "a.a-link-normal")["href"]
                except NoSuchElementException:
                    product_url = "-"

                product_data.append([brand_name, product_name, price, return_exchange, expected_delivery, availability, product_url])

        
        df = pd.DataFrame(product_data, columns=["Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability", "Product URL"])

       
        df.to_csv(f"{product_name}_search_results.csv", index=False)

    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        driver.quit()

if __name__ == "__main__":
    product_name = input("Enter the product you want to search for on Amazon.in: ")
    scrape_amazon_product_details(product_name)


In [ ]:
# 3.Q. Write a python program to access the search bar and search button on images.google.com and scrape 10 
#images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [ ]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException


def scrape_google_images(keywords, num_images=10):
    driver = None
    try:
        driver = webdriver.Chrome(executable_path='/path/to/chromedriver')  

        for keyword in keywords:
            
            driver.get("https://www.google.com/imghp")

            
            search_bar = driver.find_element_by_name("q")
            search_bar.send_keys(keyword)
            search_bar.send_keys(Keys.RETURN)

            
            time.sleep(3)

           
            for _ in range(10):
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(2)

            
            image_elements = driver.find_elements_by_css_selector(".rg_i")
            image_urls = [element.get_attribute("src") for element in image_elements if element.get_attribute("src")]

           
            os.makedirs(keyword, exist_ok=True)

            
            for i, image_url in enumerate(image_urls[:num_images], 1):
                image_filename = os.path.join(keyword, f"{keyword}_{i}.jpg")
                image_data = requests.get(image_url).content
                with open(image_filename, "wb") as f:
                    f.write(image_data)

    except WebDriverException as e:
        print(f"Selenium WebDriver Exception: {str(e)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    keywords = ['fruits', 'cars', 'Machine Learning', 'Guitar', 'Cakes']
    num_images = 10
    scrape_google_images(keywords, num_images)


In [ ]:
# 4.Q. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com
#3and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand 
#Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, 
#“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
#details is missing then replace it by “- “. Save your results in a dataframe and CSV

In [ ]:
import pandas as pd
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import warnings

warnings.filterwarnings("ignore")
import time

driver = webdriver.Chrome()
driver.get('https://www.flipkart.com')
closepopup = driver.find_element(By.XPATH, '//div[@class="JFPqaw"]/span')
closepopup.click() 

brands_field = driver.find_element(By.CLASS_NAME,"Pke_EE")
brands_field.send_keys("Oneplus Nord")
butclic = driver.find_element(By.CLASS_NAME, "_2iLD__")
butclic.click()

brand_name = []
smartphone_name = []
colour = []
ram = []
storage_rom = []
primary_camera = []
secondary_camera = []
display_size = []
battery_capacity = []
product_url = []
price = []

link = []

sub_urls =driver.find_elements(By.XPATH,'//div[@class="_2kHMtA"]/a')
for i in sub_urls:
    link.append(i.get_attribute('href'))


for i in link:
    col = []
    driver.get(i)
    time.sleep(3)
    try:
        readmoreclick=driver.find_element(By.XPATH,'//button[@class="_2KpZ6l _1FH0tX"]')
        readmoreclick.click()
    except NoSuchElementException as e:
        print("Exception Raised : ",e)
        
   
    sname=driver.find_element(By.XPATH,'//div[1][@class="_3k-BhJ"]/table/tbody/tr[3]/td[2]/ul/li')
    color=driver.find_element(By.XPATH,'//div[1][@class="_3k-BhJ"]/table/tbody/tr[4]/td[2]/ul/li')
    memory=driver.find_element(By.XPATH,'//div[4][@class="_3k-BhJ"]/table/tbody/tr[2]/td[2]/ul/li')
    rom=driver.find_element(By.XPATH,'//div[4][@class="_3k-BhJ"]/table/tbody/tr[1]/td[2]/ul/li')
    pcam=driver.find_element(By.XPATH,'//div[5][@class="_3k-BhJ"]/table/tbody/tr[1]/td[2]/ul/li')
    scam=driver.find_element(By.XPATH,'//div[5][@class="_3k-BhJ"]/table/tbody/tr[2]/td[2]/ul/li')
    #pdisplay=driver.find_element(By.XPATH,'//div[2][@class="_3k-BhJ"]/table/tbody/tr[1]/td[2]/ul/li')
    #bat=driver.find_element(By.XPATH,'//div[7][@class="_3k-BhJ"]/table/tbody/tr[1]/td[2]/ul/li')
    #pri=driver.find_element(By.CLASS_NAME,"_30jeq3 _16Jk6d")
    
    smartphone_name.append(sname.text)
    colour.append(color.text)
    ram.append(memory.text)
    storage_rom.append(rom.text)
    primary_camera.append(pcam.text)
    secondary_camera.append(scam.text)
    #display_size.append(pdisplay.text)
    #battery_capacity.append(bat.text)
    product_url.append(i)
    #price.append(pri.text)
    

print(len(smartphone_name),len(colour),len(ram))
df = pd.DataFrame({'smartphone_name':smartphone_name ,'colour':colour,'ram':ram,'storage_rom':storage_rom})
df
    



In [ ]:
# 5.Q.Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [ ]:

import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
import warnings

warnings.filterwarnings("ignore")


def scrape_coordinates(city_name):
    driver = None
    try:
        driver = webdriver.Chrome()  
        driver.get("https://www.google.com/maps")
        time.sleep(3) 

       
        search_bar = driver.find_element_by_id("searchboxinput")
        search_bar.clear()
        search_bar.send_keys(city_name)
        search_bar.send_keys('\n')
        time.sleep(3)  
        
        
        url = driver.current_url
        coordinates = url.split("@")[1].split(",")[0:2]
        latitude, longitude = coordinates
        print(f"City: {city_name}")
        print(f"Latitude: {latitude}")
        print(f"Longitude: {longitude}")

    except WebDriverException as e:
        print(f"Selenium WebDriver Exception: {str(e)}")
    except Exception as e:
        print(f"An error occurred: {str(e)}")
    finally:
        if driver:
            driver.quit()

if __name__ == "__main__":
    city_name = input("Enter the city name to get geospatial coordinates: ")
    scrape_coordinates(city_name)


In [ ]:
# Q.6. Write a program to scrap all the available details of best gaming laptops from digit.in. 

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()

driver.get('https://www.digit.in/')


laptops_link = driver.find_element(By.XPATH, '//a[contains(@href, "laptops")]')
laptops_link.click()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="searchPage"]')))

best_gaming_laptops_link = driver.find_element(By.XPATH, '//a[contains(@href, "best-gaming-laptops")]')
best_gaming_laptops_link.click()

WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//div[@class="searchPage"]')))


laptop_names = driver.find_elements(By.XPATH, '//div[@class="TopNumbeHeading active sticky-footer"]/a')
for laptop_name in laptop_names:
    print(laptop_name.text)


driver.quit()


In [ ]:
#Q.7. Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
#“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [8]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()


driver.get('https://www.forbes.com/billionaires/')


WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'table-body')))

billionaires_data = []
rows = driver.find_elements(By.XPATH, '//div[@class="table-body"]/div')
for row in rows:
    rank = row.find_element(By.XPATH, './/div[@class="rank"]').text.strip()
    name = row.find_element(By.XPATH, './/div[@class="personName"]').text.strip()
    net_worth = row.find_element(By.XPATH, './/div[@class="netWorth"]').text.strip()
    age = row.find_element(By.XPATH, './/div[@class="age"]').text.strip()
    citizenship = row.find_element(By.XPATH, './/div[@class="countryOfCitizenship"]').text.strip()
    source = row.find_element(By.XPATH, './/div[@class="source"]').text.strip()
    industry = row.find_element(By.XPATH, './/div[@class="category"]').text.strip()

    billionaire_details = {
        "Rank": rank,
        "Name": name,
        "Net worth": net_worth,
        "Age": age,
        "Citizenship": citizenship,
        "Source": source,
        "Industry": industry
    }

    billionaires_data.append(billionaire_details)


for billionaire in billionaires_data:
    print(billionaire)


driver.quit()


TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6F74B82B2+55298]
	(No symbol) [0x00007FF6F7425E02]
	(No symbol) [0x00007FF6F72E05AB]
	(No symbol) [0x00007FF6F732175C]
	(No symbol) [0x00007FF6F73218DC]
	(No symbol) [0x00007FF6F735CBC7]
	(No symbol) [0x00007FF6F73420EF]
	(No symbol) [0x00007FF6F735AAA4]
	(No symbol) [0x00007FF6F7341E83]
	(No symbol) [0x00007FF6F731670A]
	(No symbol) [0x00007FF6F7317964]
	GetHandleVerifier [0x00007FF6F7830AAB+3694587]
	GetHandleVerifier [0x00007FF6F788728E+4048862]
	GetHandleVerifier [0x00007FF6F787F173+4015811]
	GetHandleVerifier [0x00007FF6F75547D6+695590]
	(No symbol) [0x00007FF6F7430CE8]
	(No symbol) [0x00007FF6F742CF34]
	(No symbol) [0x00007FF6F742D062]
	(No symbol) [0x00007FF6F741D3A3]
	BaseThreadInitThunk [0x00007FFADE80257D+29]
	RtlUserThreadStart [0x00007FFADF24AA58+40]


In [ ]:
#Q. 8. Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video.

In [ ]:
#Q.9.Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
#“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
#reviews, privates from price, dorms from price, facilities and property description.

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC


driver = webdriver.Chrome()


driver.get('https://www.hostelworld.com/')


search_box = driver.find_element(By.ID, 'home-search-keywords')
search_box.send_keys('London')
search_box.submit()


WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 'result')))


hostels_data = []
hostel_cards = driver.find_elements(By.CLASS_NAME, 'result')
for card in hostel_cards:
    hostel_name = card.find_element(By.CLASS_NAME, 'title').text
    distance_from_city_center = card.find_element(By.CLASS_NAME, 'distanceFromCenter').text
    ratings = card.find_element(By.CLASS_NAME, 'score').text
    total_reviews = card.find_element(By.CLASS_NAME, 'reviews').text
    overall_reviews = card.find_element(By.CLASS_NAME, 'keyword').text
    privates_from_price = card.find_element(By.CLASS_NAME, 'price').text
    dorms_from_price = card.find_element(By.CLASS_NAME, 'price').text

   
    card.click()

    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'hw-special-points')))

    
    facilities = driver.find_element(By.ID, 'hw-special-points').text
    property_description = driver.find_element(By.CLASS_NAME, 'description').text

    hostel_details = {
        "Hostel Name": hostel_name,
        "Distance from City Centre": distance_from_city_center,
        "Ratings": ratings,
        "Total Reviews": total_reviews,
        "Overall Reviews": overall_reviews,
        "Privates from Price": privates_from_price,
        "Dorms from Price": dorms_from_price,
        "Facilities": facilities,
        "Property Description": property_description
    }

    hostels_data.append(hostel_details)

   
    driver.back()

for hostel in hostels_data:
    print(hostel)


driver.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="home-search-keywords"]"}
  (Session info: chrome=119.0.6045.160); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6F74B82B2+55298]
	(No symbol) [0x00007FF6F7425E02]
	(No symbol) [0x00007FF6F72E05AB]
	(No symbol) [0x00007FF6F732175C]
	(No symbol) [0x00007FF6F73218DC]
	(No symbol) [0x00007FF6F735CBC7]
	(No symbol) [0x00007FF6F73420EF]
	(No symbol) [0x00007FF6F735AAA4]
	(No symbol) [0x00007FF6F7341E83]
	(No symbol) [0x00007FF6F731670A]
	(No symbol) [0x00007FF6F7317964]
	GetHandleVerifier [0x00007FF6F7830AAB+3694587]
	GetHandleVerifier [0x00007FF6F788728E+4048862]
	GetHandleVerifier [0x00007FF6F787F173+4015811]
	GetHandleVerifier [0x00007FF6F75547D6+695590]
	(No symbol) [0x00007FF6F7430CE8]
	(No symbol) [0x00007FF6F742CF34]
	(No symbol) [

In [1]:
print("hu")

hu
